In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import regularizers

In [2]:
# Set the input shape
INPUT_SHAPE = (224, 224, 3)

# Define the number of classes
NUM_CLASSES = 8

# Define the batch size
BATCH_SIZE = 32

# Define the number of epochs
EPOCHS = 50

# Define the paths to the train, validation, and test directories
TRAIN_DIR = "/kaggle/input/preprocessed-dataset/dataset-split/train"
VAL_DIR = "/kaggle/input/preprocessed-dataset/dataset-split/val"
TEST_DIR = "/kaggle/input/preprocessed-dataset/dataset-split/test"

# Define the data augmentation parameters
data_augmentation = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

# Define the training dataset generator
train_generator = data_augmentation.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

for i in train_generator:
    print(i[1])
    break

# Define the validation dataset generator
val_generator = ImageDataGenerator().flow_from_directory(
    directory=VAL_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# Define the test dataset generator
test_generator = ImageDataGenerator().flow_from_directory(
    directory=TEST_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# Load the ResNet50 model pre-trained on ImageNet
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_output = resnet.layers[-1].output
resnet_output = layers.GlobalAveragePooling2D()(resnet_output)
resnet_output = layers.Reshape((1, 1, 2048))(resnet_output)
resnet_model = Model(inputs=resnet.input, outputs=resnet_output)

# Set the ResNet layers to be non-trainable
for layer in resnet.layers[:-13]:
    layer.trainable = False

# Define the ResNet-50 model
model = keras.Sequential([
    resnet_model,  # Added the ResNet50 model here
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation="softmax")
])

# Compile the model with a learning rate of 0.001
opt = keras.optimizers.Adam(learning_rate=0.0002)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

Found 3960 images belonging to 8 classes.
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]
Found 1130 images belonging to 8 classes.
Found 575 images belonging to 8 classes.
94765736/9

In [3]:
print(model.summary())
from keras.callbacks import ModelCheckpoint
filepath = "/kaggle/working/CNN.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1, 1, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

In [4]:
# train model
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.n // train_generator.batch_size, 
    epochs=50, 
    validation_data=val_generator, 
    validation_steps=val_generator.n // val_generator.batch_size,
    callbacks=callbacks_list)

Epoch 1/50
123/123 [==============================] - ETA: 0s - loss: 10.8816 - accuracy: 0.1800
Epoch 1: loss improved from inf to 10.88159, saving model to /kaggle/working/CNN.h5
123/123 [==============================] - 94s 672ms/step - loss: 10.8816 - accuracy: 0.1800 - val_loss: 9.8858 - val_accuracy: 0.2571
Epoch 2/50
123/123 [==============================] - ETA: 0s - loss: 9.9416 - accuracy: 0.2485
Epoch 2: loss improved from 10.88159 to 9.94159, saving model to /kaggle/working/CNN.h5
123/123 [==============================] - 71s 580ms/step - loss: 9.9416 - accuracy: 0.2485 - val_loss: 9.6380 - val_accuracy: 0.3384
Epoch 3/50
123/123 [==============================] - ETA: 0s - loss: 9.1893 - accuracy: 0.2839
Epoch 3: loss improved from 9.94159 to 9.18929, saving model to /kaggle/working/CNN.h5
123/123 [==============================] - 74s 602ms/step - loss: 9.1893 - accuracy: 0.2839 - val_loss: 8.7303 - val_accuracy: 0.2973
Epoch 4/50
123/123 [=============================

In [5]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

from sklearn.metrics import classification_report
# Generate predictions on the test set
y_pred = model.predict(test_generator)
# Convert predictions from one-hot encoded to class labels
y_pred = tf.argmax(y_pred, axis=1)
# Generate true labels for the test set
y_true = test_generator.classes
# Generate a classification report
report = classification_report(y_true, y_pred)
# Print the classification report
print(report)

18/18 [==============================] - 7s 381ms/step - loss: 1.5062 - accuracy: 0.5426
Test loss: 1.5061990022659302
Test accuracy: 0.5426086783409119
18/18 [==============================] - 5s 216ms/step
              precision    recall  f1-score   support

           0       0.65      0.71      0.68        91
           1       0.67      0.63      0.65        97
           2       0.91      0.24      0.38        42
           3       0.58      0.46      0.52        82
           4       0.51      0.44      0.47        89
           5       0.47      0.39      0.42        44
           6       0.40      0.52      0.45        94
           7       0.46      0.92      0.61        36

    accuracy                           0.54       575
   macro avg       0.58      0.54      0.52       575
weighted avg       0.57      0.54      0.54       575

